In [1]:
import time
from timeit import default_timer
from contextlib import contextmanager

@contextmanager
def elapsed_timer():
    start = default_timer()
    print 'I\'m here.' + time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime())
    elapser = lambda: default_timer() - start
    yield lambda: elapser()
    end = default_timer()
    elapser = lambda: end-start
    print 'I\'m done.' + time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime())

In [2]:
with elapsed_timer() as elapsed:
    time.sleep(1)
    duration = '%.1f' % elapsed()
    print duration
    time.sleep(4)

I'm here.2015-12-09 12:47:26
1.0
I'm done.2015-12-09 12:47:31


In [65]:
%%bash
#Split the all_views, get one file(sentiment, tweet) for each view
#Clean each view
#Run this for each view.
norm() {
    fn=$1
    if [ ! -f "$fn" ]
    then
        echo "File: $fn not found"
        exit
    fi
    #this function will convert text to lowercase and will disconnect punctuation and special symbols from words
    function normalize_text {
        awk '{print tolower($0);}' < $1 | sed -e 's/\./ \. /g' -e 's/<br \/>/ /g' -e 's/"/ " /g' \
        -e 's/,/ , /g' -e 's/(/ ( /g' -e 's/)/ ) /g' -e 's/\!/ \! /g' -e 's/\?/ \? /g' \
        -e 's/\;/ \; /g' -e 's/\:/ \: /g' > $1-norm
    }
    export LC_ALL=C
    normalize_text "$fn"
    wc -l $fn
    mv "$fn" "$fn-norm"
}
norm "data/view1_clean" #file name is
norm "data/view2_clean" 

    3137 data/view1_clean
    3137 data/view2_clean


In [69]:
import os.path
tw_view_1 = 'data/view1_clean-norm'
tw_view_2 = 'data/view2_clean-norm'
assert os.path.isfile(tw_view_1), tw_view_1 + " unavailable"
assert os.path.isfile(tw_view_2), tw_view_2 + " unavailable"

In [70]:
import gensim
import numpy as np
from gensim.models.doc2vec import TaggedDocument
from collections import namedtuple, defaultdict as dd

#sentiment = {'positive':1, 'negative':-1} #, 'neutral':2}
sentiment_dict = {'4':1, '0':-1} #- new data 0,4
SentimentDocument = namedtuple('SentimentDocument', 'words tags split sentiment')

alldocs = dd(list)  # will hold all docs in original order - dictionary, keys = [v1, v2]
v1 = 'view1'
v2 = 'view2'
#tw_sentiment_dict = {}
#print total_num, train_test_shuffle
all_v2_words = []
with open(tw_view_2) as allview2:
        all_v2_words = allview2.readlines()
total_num = len(all_v2_words)
#split train/test
train_num = total_num *  7 / 10 # 70% train/test
train_test_shuffle = np.arange(total_num)
np.random.shuffle(train_test_shuffle)
with open(tw_view_1) as allview1:
    #for line_no, (v1, v2) in enumerate(zip(allview1, allview2)):
    for line_no, line in enumerate(allview1):
        tokens = gensim.utils.to_unicode(line).split('\t')
        if len(tokens) != 2:
            print line
            raise Exception()
        sentiment = sentiment_dict[tokens[0]]
        #if tw_id not in tw_sentiment_dict.keys():
        #    continue
        words = tokens[1]
        split = 'train' if train_test_shuffle[line_no] <= train_num else 'test'
        #sentiment = tw_sentiment_dict[tw_id]
        v2_words = gensim.utils.to_unicode(all_v2_words[line_no]).split('\t')[1]
        
        alldocs[v1].append(SentimentDocument(words, [line_no], split, sentiment))
        alldocs[v2].append(SentimentDocument(v2_words, [line_no], split, sentiment))
train_docs = {
    v1 : [doc for doc in alldocs[v1] if doc.split == 'train'],
    v2 : [doc for doc in alldocs[v2] if doc.split == 'train']
}
test_docs = {
    v1 : [doc for doc in alldocs[v1] if doc.split == 'test'],
    v2 : [doc for doc in alldocs[v2] if doc.split == 'test']
}
doc_list = { v1: alldocs[v1][:], v2: alldocs[v2][:] }  # for reshuffling per pass

print('%d docs: %d train-sentiment, %d test-sentiment' % (len(doc_list[v1]), len(train_docs[v1]), len(test_docs[v1])))

3137 docs: 2196 train-sentiment, 941 test-sentiment


In [125]:
from gensim.models import Doc2Vec
import gensim.models.doc2vec
from collections import OrderedDict
import multiprocessing

cores = multiprocessing.cpu_count()
assert gensim.models.doc2vec.FAST_VERSION > -1, "this will be painfully slow otherwise"
model_size = 200
simple_models , models_by_name = {}, {} 
for view in [v1, v2]:
    simple_models[view] = [
        # PV-DM w/concatenation - window=5 (both sides) approximates paper's 10-word total window size
        Doc2Vec(dm=1, dm_concat=1, size=model_size, window=3, negative=5, hs=0, min_count=2, workers=cores),
        # PV-DBOW 
        Doc2Vec(dm=0, size=model_size, negative=5, hs=0, min_count=5, workers=cores),
        # PV-DM w/average
        Doc2Vec(dm=1, dm_mean=1, size=model_size, window=3, negative=5, hs=0, min_count=2, workers=cores),
    ]

    # speed setup by sharing results of 1st model's vocabulary scan
    simple_models[view][0].build_vocab(alldocs[view])  # PV-DM/concat requires one special NULL word so it serves as template
    print view, simple_models[view][0]
    for model in simple_models[view][1:]:
        model.reset_from(simple_models[view][0])
        print view, model

    models_by_name[view] = OrderedDict((str(model), model) for model in simple_models[view])

view1 Doc2Vec(dm/c,d200,n5,w3,mc2,t4)
view1 Doc2Vec(dbow,d200,n5,mc5,t4)
view1 Doc2Vec(dm/m,d200,n5,w3,mc2,t4)
view2 Doc2Vec(dm/c,d200,n5,w3,mc2,t4)
view2 Doc2Vec(dbow,d200,n5,mc5,t4)
view2 Doc2Vec(dm/m,d200,n5,w3,mc2,t4)


In [126]:
from gensim.test.test_doc2vec import ConcatenatedDoc2Vec
for view in [v1, v2]:
    models_by_name[view]['dbow+dmm'] = ConcatenatedDoc2Vec([simple_models[view][1], simple_models[view][2]])
    models_by_name[view]['dbow+dmc'] = ConcatenatedDoc2Vec([simple_models[view][1], simple_models[view][0]])
#print models_by_name['dbow+dmm'], models_by_name['dbow+dmc'] 
#del models_by_name['dbow+dmc']

In [136]:
import numpy as np
import statsmodels.api as sm
from sklearn import svm, metrics, neighbors
from random import sample

# for timing
from contextlib import contextmanager
from timeit import default_timer
import time 
import ipdb

@contextmanager
def elapsed_timer():
    start = default_timer()
    elapser = lambda: default_timer() - start
    yield lambda: elapser()
    end = default_timer()
    elapser = lambda: end-start
    
def logistic_predictor_from_data(train_targets, train_regressors):
    logit = sm.Logit(train_targets, train_regressors)
    #ipdb.set_trace()
    predictor = logit.fit(disp=0)
    #print(predictor.summary())
    return predictor

def svm_predictor_from_data(train_targets, train_regressors):
    svc = svm.SVC(kernel='rbf', degree=5, gamma=1e-3)
    svc.fit(train_regressors, train_targets)
    return svc

    """expected = svm_y_test
    predicted = svc.predict(svm_x_test)

    #print("Classification report for classifier %s:\n%s\n"
    #      % (svc, metrics.classification_report(expected, predicted)))
    #print("Confusion matrix:\n%s" % metrics.confusion_matrix(expected, predicted))
    """
    
def knn_predictor_from_data(train_targets, train_regressors):
    k = 5
    clf = neighbors.KNeighborsClassifier(k)
    clf.fit(train_regressors, train_targets)
    return clf

def error_rate_for_model(test_model, train_set, test_set, infer=False, infer_steps=3, infer_alpha=0.1, infer_subsample=0.1):
    """Report error rate on test_doc sentiments, using supplied model and train_docs"""

    train_targets, train_regressors = zip(*[(doc.sentiment, test_model.docvecs[doc.tags[0]]) for doc in train_set])
    #train_regressors = sm.add_constant(train_regressors)
    #predictor = logistic_predictor_from_data(train_targets, train_regressors)
    predictor = svm_predictor_from_data(train_targets, train_regressors)
    #predictor = knn_predictor_from_data(train_targets, train_regressors)

    test_data = test_set
    if infer:
        if infer_subsample < 1.0:
            test_data = sample(test_data, int(infer_subsample * len(test_data)))
        test_regressors = [test_model.infer_vector(doc.words, steps=infer_steps, alpha=infer_alpha) for doc in test_data]
    else:
        test_regressors = [test_model.docvecs[doc.tags[0]] for doc in test_data]
    #test_regressors = sm.add_constant(test_regressors)
    
    # predict & evaluate
    test_predictions = predictor.predict(test_regressors)
    predicted = np.rint(test_predictions)
    expected = [doc.sentiment for doc in test_data]
    """if not infer:
        print("Classification report for classifier %s:\n%s\n"
              % (predictor, metrics.classification_report(expected, predicted)))
        print("Confusion matrix:\n%s" % metrics.confusion_matrix(expected, predicted))"""
    #ipdb.set_trace()
    corrects = sum(expected == predicted)
    errors = len(test_predictions) - corrects
    error_rate = float(errors) / len(test_predictions)
    return (error_rate, errors, len(test_predictions), predictor)

In [137]:
#from collections import defaultdict
best_error = dd(lambda: dd(lambda :(1.0, 0.0)))  # to selectively-print only best errors achieved

In [ ]:
print 'Started.'
from random import shuffle
import datetime

for view in [v1, v2]:
    alpha, min_alpha, passes = (0.025, 0.001, 20)
    alpha_delta = (alpha - min_alpha) / passes

    print "======== %s =========" %view
    print("START %s" % datetime.datetime.now())

    for epoch in range(passes):
        shuffle(doc_list[view])  # shuffling gets best results

        for name, train_model in models_by_name[view].items():
            #print name
            # train
            duration = 'na'
            train_model.alpha, train_model.min_alpha = alpha, alpha
            with elapsed_timer() as elapsed:
                train_model.train(doc_list[view])
                duration = '%.1f' % elapsed()

            # evaluate
            eval_duration = ''
            with elapsed_timer() as eval_elapsed:
                err, err_count, test_count, predictor = error_rate_for_model(train_model, train_docs[view], test_docs[view])
            eval_duration = '%.1f' % eval_elapsed()
            best_indicator = ' '
            if err <= best_error[view][name][0]:
                best_error[view][name] = (err, alpha)
                best_indicator = '*' 
            print("%s%f : %i passes : %s-%s %ss %ss" % (best_indicator, err, epoch + 1, view, name, duration, eval_duration))

            if ((epoch + 1) % 5) == 0 or epoch == 0:
                eval_duration = ''
                with elapsed_timer() as eval_elapsed:
                    infer_err, err_count, test_count, predictor = error_rate_for_model(train_model, train_docs[view], test_docs[view], infer=True)
                eval_duration = '%.1f' % eval_elapsed()
                best_indicator = ' '
                if infer_err < best_error[view][name + '_inferred'][0]:
                    best_error[view][name + '_inferred'] = (infer_err, alpha)
                    best_indicator = '*'
                print("%s%f : %i passes : %s-%s %ss %ss" % (best_indicator, infer_err, epoch + 1, view, name + '_inferred', duration, eval_duration))

        print('completed pass %i at alpha %f' % (epoch + 1, alpha))
        alpha -= alpha_delta

    print("END %s" % str(datetime.datetime.now()))

Started.
======== view1 =========
START 2015-12-10 11:48:17.333701
*0.448459 : 1 passes : view1-Doc2Vec(dm/c,d200,n5,w3,mc2,t4) 0.5s 1.8s
*0.500000 : 1 passes : view1-Doc2Vec(dm/c,d200,n5,w3,mc2,t4)_inferred 0.5s 1.5s
*0.448459 : 1 passes : view1-Doc2Vec(dbow,d200,n5,mc5,t4) 0.2s 1.7s
*0.351064 : 1 passes : view1-Doc2Vec(dbow,d200,n5,mc5,t4)_inferred 0.2s 1.4s
*0.447396 : 1 passes : view1-Doc2Vec(dm/m,d200,n5,w3,mc2,t4) 0.3s 1.8s
*0.468085 : 1 passes : view1-Doc2Vec(dm/m,d200,n5,w3,mc2,t4)_inferred 0.3s 1.3s
*0.428268 : 1 passes : view1-dbow+dmm 0.0s 3.2s
*0.531915 : 1 passes : view1-dbow+dmm_inferred 0.0s 2.5s
*0.448459 : 1 passes : view1-dbow+dmc 0.0s 3.2s
*0.489362 : 1 passes : view1-dbow+dmc_inferred 0.0s 2.5s
completed pass 1 at alpha 0.025000
*0.448459 : 2 passes : view1-Doc2Vec(dm/c,d200,n5,w3,mc2,t4) 0.5s 1.6s
*0.448459 : 2 passes : view1-Doc2Vec(dbow,d200,n5,mc5,t4) 0.2s 1.6s
*0.447396 : 2 passes : view1-Doc2Vec(dm/m,d200,n5,w3,mc2,t4) 0.3s 1.7s
 0.429330 : 2 passes : view1-db

In [ ]:
for view in [v1, v2]:
    print '========= %s ========' %view
    for rate, alpha, name in sorted((rate, alpha, name) for name, (rate, alpha) in best_error[view].items()):
        print("%f %s %f" % (rate, name, alpha))

In [87]:
# Print some example tweets and their vector reps for both views
index = 70
print alldocs['view1'][index]
tag = alldocs['view1'][index].tags[0]
print '\n', simple_models['view1'][0].docvecs[tag]

print '\n', alldocs['view2'][index]
print '\n', simple_models['view2'][0].docvecs[tag]
#print '\n\n', doc_list['view2'][:10]

SentimentDocument(words=u"||T|| Can't wait to see you in 4 sleeps and 3 days!\n", tags=[70], split='train', sentiment=1)

[ -4.25896272e-02  -7.51020536e-02   4.90097255e-02   1.93141222e-01
  -3.93390395e-02  -2.54712440e-02  -1.70356892e-02  -1.23695590e-01
   1.25351757e-01  -2.56510638e-03  -2.95490623e-02  -1.99460816e-02
  -4.66628447e-02  -9.13957953e-02  -9.39208269e-02   4.74906079e-02
  -3.71774212e-02   5.00000305e-02  -5.97346341e-03  -2.45082304e-02
   2.24953853e-02   1.57746542e-02   9.52238142e-02  -1.06800541e-01
  -1.63270339e-01   8.37060958e-02   3.07731070e-02   4.16026153e-02
  -5.64083755e-02  -1.10432275e-01  -6.92696944e-02  -1.12972274e-01
   1.35694817e-01   1.09122008e-01  -9.25321039e-03   2.34711356e-02
   1.41892079e-02   1.17532341e-02   1.84597040e-04   1.59151733e-01
  -5.66027574e-02   1.16361395e-01   3.45780104e-02  -2.97031198e-02
  -1.41879460e-02  -9.31217298e-02   2.26680897e-02  -9.47654545e-02
   1.54242486e-01  -8.31641927e-02   2.46877559e-0

In [ ]:
doc_id = np.random.randint(simple_models[0].docvecs.count)  # pick random doc; re-run cell for more examples
print('for doc %d...' % doc_id)
for model in simple_models:
    inferred_docvec = model.infer_vector(alldocs[doc_id].words)
    print('%s:\n %s' % (model, model.docvecs.most_similar([inferred_docvec], topn=3)))

In [131]:
def center(data):
    return data - np.mean(data, axis=0)

def PLS(X, Y):
    cross_cov = np.dot(center(X), center(Y).T)
    eigval,eigvec=np.linalg.eig(cross_cov.dot(cross_cov.T))
    return (eigval, eigvec)

def PLS_MFCC():
    dims = [10, 30, 50, 70, 90, 110]
    accuracies = np.zeros((len(num_neighb), len(dims)))
    # (score, dim, k, PLS_subspace, classifier_object)
    best = (0, 0, 0, None, None) 
    #run pls
    eigval, U = PLS(acoustic_train, artic_train)
    for j, k in enumerate(num_neighb):
        for i, d in enumerate(dims):
            U_d = get_top_eigvec(eigval, U, d)
            #get projection to pls space
            train_proj = np.dot(U_d.T, acoustic_train_cen)
            dev_proj = np.dot(U_d.T, acoustic_dev_cen)
            # stack with mfcc39
            stacked_train = np.append(train_proj, mfcc39_train, axis=0)
            stacked_dev = np.append(dev_proj, mfcc39_dev, axis=0)
            
            #classify
            clf = neighbors.KNeighborsClassifier(k)
            clf.fit(stacked_train.T, phones_train)

            #predictions
            score = clf.score(stacked_dev.T, phones_dev)
            if score > best[0]:
                best = (score, d, k, U_d, clf)
            accuracies[j,i] = score
    return (best, accuracies)

def CCA(X, Y, regX = 0, regY = 0):
    cenX = center(X)
    cenY = center(Y)
    cross_cov = cenX.dot(cenY.T)
    covX = cenX.dot(cenX.T)
    covY = cenY.dot(cenY.T)
    r_Ix = regX * np.eye(covX.shape[0])
    r_Iy = regY * np.eye(covY.shape[0])
    A = reduce(np.dot, [ np.linalg.inv(covX + r_Ix), cross_cov, np.linalg.inv(covY + r_Iy), cross_cov.T ])
    eigval,eigvec=np.linalg.eig(A)
    return (eigval, eigvec)

def get_top_eigvec(eigval, eigvec, k):
    idx=np.argsort(eigval)[-k:][::-1]
    #eigval=eigval[idx]
    return eigvec[:,idx]

def CCA_MFCC():
    dims = [10, 30, 50, 70, 90, 110]
    reg = [1e-8, 1e-6, 1e-4, 1e-2, 1e-1, 1e1]
    accuracies = np.zeros((len(reg), len(reg), len(dims), len(num_neighb)))
    # (score, dim, regX, regY, k, CCA_subspace, classifier_object)
    best = (0, 0, 0, 0, 0, None, None)
    for rx, regX in enumerate(reg):
        for ry, regY in enumerate(reg):              
            #run cca
            eigval, U = CCA(acoustic_train, artic_train, regX, regY)
            for i, d in enumerate(dims):
                U_d = get_top_eigvec(eigval, U, d)
                #get projection to cca space
                train_proj = U_d.T.dot(acoustic_train_cen)
                dev_proj = U_d.T.dot(acoustic_dev_cen)
                # stack with mfcc39
                stacked_train = np.append(train_proj, mfcc39_train, axis=0)
                stacked_dev = np.append(dev_proj, mfcc39_dev, axis=0)
                #classify
                for j, k in enumerate(num_neighb):
                    clf = neighbors.KNeighborsClassifier(k)
                    clf.fit(stacked_train.T, phones_train)

                    #predictions
                    score = clf.score(stacked_dev.T, phones_dev)
                    if score > best[0]:
                        best = (score, d, regX, regY, k, U_d, clf)
                    accuracies[rx, ry, i, j] = score
    return (best, accuracies)

def plot_pc2(data, eigvec, phones_data):
    #project to top 2 princ. comp.
    data_proj = np.dot(np.transpose(eigvec), data)
    data_proj_labels=[data_proj[:,np.where(phones_data==lbl)] for lbl in labels_dict.values()]
    #Plot
    cmap = plt.get_cmap('jet_r')
    N=len(labels)
    colors = [cmap(float(i)/N) for i in np.linspace(5.0, 0, N)]
    plt.figure(figsize=(7,7))
    #plt.subplot(2,1,1)
    for i in range(N):
        plt.scatter(data_proj_labels[i][0,:], data_proj_labels[i][1,:] ,c=colors[i], marker='+', label=labels[i]);
    #plt.legend(plots,labels)
    plt.legend(loc=3)
    #plt.show()
    return plt

In [ ]:
#Select the best performing word2vec model
best_model_name = 'dbow+dmc'
best_alpha = 0.0022000
best_model = { v1 : models_by_name[v1][best_model_name],
              v2 : models_by_name[v2][best_model_name] }
# Train best model
shuffle(doc_list[view])
for view in [v1, v2]:
    best_model[v1].alpha, best_model[v1].min_alpha = best_alpha, best_alpha
    best_model[v1].train(doc_list[view])

In [ ]:
 # DO CCA on the training docvecs
# X = view 1, Y = view 2 : [num_samples x word_vec_size]
target_sentiments, X, Y = zip(*[(doc.sentiment, best_model[v1].docvecs[doc.tags[0]], \
                             best_model[v2].docvecs[doc.tage[0]]) for doc in train_set])
(cca_eigval, cca_eigvec) = CCA(X.T, Y.T)